In [1]:
%cd /home/brimmann/works/llm-recipes

/home/brimmann/works/llm-recipes


/home/brimmann/works/llm-recipes/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from data.data_utils import get_dataloader
from configs.datasets import dataset as DatasetConfig
from configs.training import train_config as TrainConfig
from configs.distillation import distillation_config as DistillationConfig

In [3]:
dataset_config = DatasetConfig()
train_config = TrainConfig()
distill_config = DistillationConfig()
train_config.model_name = "EleutherAI/pythia-70m-deduped"
distill_config.model_name = "mistralai/Mistral-7B-Instruct-v0.2"
train_config.batch_size_training = 1
train_config.num_workers_dataloader = 1
dataset_config.file = "datasets_m/loader/squad.py"
dataset_config.xrag = False

In [4]:
from transformers import AutoTokenizer
student_tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
teacher_tokenizer = AutoTokenizer.from_pretrained(distill_config.model_name)
student_tokenizer.pad_token_id = student_tokenizer.eos_token_id
teacher_tokenizer.pad_token_id = teacher_tokenizer.eos_token_id

In [5]:
student_train_dldr, student_eval_dldr = get_dataloader(dataset_config, train_config, student_tokenizer, 0, distill_config)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Few shot is zero
Few shot is zero
Few shot is zero
--> Training Set Length = 3


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Few shot is zero
--> Validation Set Length = 1


In [6]:
next(iter(student_train_dldr)).keys()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


dict_keys(['input_ids', 'attention_mask', 'labels'])

In [6]:
next(iter(student_train_dldr)).keys()

dict_keys(['input_ids', 'attention_mask', 'retrieval_embeds', 'labels'])

In [ ]:
teacher_train_dldr, teacher_eval_dldr = get_dataloader(dataset_config, train_config, teacher_tokenizer, 0, distill_config)

In [ ]:
len(teacher_train_dldr)

In [ ]:
itr1 = iter(teacher_train_dldr)
itr2 = iter(student_train_dldr)

In [ ]:
next(itr1).keys()

In [ ]:
print(teacher_tokenizer.decode(next(itr1)["input_ids"][0], skip_special_tokens=False))

In [ ]:
print(student_tokenizer.decode(next(itr2)["input_ids"][0], skip_special_tokens=False))